In [2]:
import torchvision
import torch
import os
import random
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [3]:
from torchvision.datasets.utils import download_and_extract_archive
from torchvision.datasets.folder import ImageFolder

In [4]:
data_transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225])
])

In [5]:
if not os.path.exists("/home/23dcs505/data/2750"):
    print("No dataset found")
fulldata=ImageFolder(root='/home/23dcs505/data/2750', transform=data_transform)


In [6]:
from torch.utils.data import random_split

train_len=int((0.8)*len(fulldata))
test_len=len(fulldata)-(train_len)

train_data_set,test_data_set= random_split(fulldata,[train_len, test_len])

In [7]:
all_list=[0,1,2,3,4,5,6,7,8,9]

In [8]:
train_class_len=8

In [9]:
train_list=random.sample(all_list,train_class_len)
test_list=list(range(0,10))
strict_test_list=list(set(all_list) - set(train_list))


In [10]:
print(train_list)
print(test_list)
print(strict_test_list)

[6, 2, 1, 3, 8, 9, 5, 7]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 4]


In [37]:
ways=5
shots=3
queries=5
strict_ways=len(strict_test_list)

In [12]:
from torch.utils.data import Subset

In [13]:
train_data_set.indices[0]

15063

In [14]:
def class_sorting(dataset, class_list):
    targets = dataset.dataset.targets

    indices= [i for i in dataset.indices if targets[i] in class_list]
    return Subset(dataset.dataset, indices)
    

In [15]:
train_data=class_sorting(train_data_set,train_list)
test_data=class_sorting(test_data_set,test_list)
strict_test_data=class_sorting(test_data,strict_test_list)

In [16]:
train_data[0]

(tensor([[[-1.0219, -1.0219, -1.0219,  ..., -0.6965, -0.6965, -0.6965],
          [-1.0219, -1.0219, -1.0219,  ..., -0.6965, -0.6965, -0.6965],
          [-1.0219, -1.0219, -1.0219,  ..., -0.6965, -0.6965, -0.6965],
          ...,
          [-1.1418, -1.1418, -1.1418,  ..., -1.0390, -1.0390, -1.0390],
          [-1.1418, -1.1418, -1.1418,  ..., -1.0390, -1.0390, -1.0390],
          [-1.1418, -1.1418, -1.1418,  ..., -1.0390, -1.0390, -1.0390]],
 
         [[-0.2500, -0.2500, -0.2500,  ..., -0.0049, -0.0049, -0.0049],
          [-0.2500, -0.2500, -0.2500,  ..., -0.0049, -0.0049, -0.0049],
          [-0.2500, -0.2500, -0.2500,  ..., -0.0049, -0.0049, -0.0049],
          ...,
          [-0.3901, -0.3901, -0.3901,  ..., -0.4426, -0.4426, -0.4426],
          [-0.3901, -0.3901, -0.3901,  ..., -0.4426, -0.4426, -0.4426],
          [-0.3901, -0.3901, -0.3901,  ..., -0.4426, -0.4426, -0.4426]],
 
         [[ 0.3742,  0.3742,  0.3742,  ...,  0.5136,  0.5136,  0.5136],
          [ 0.3742,  0.3742,

In [17]:

train_data.dataset

Dataset ImageFolder
    Number of datapoints: 27000
    Root location: /home/23dcs505/data/2750
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [18]:
train_data.indices[0]

15063

In [19]:
from torch.utils.data import Dataset

class create_dataset(Dataset):
    def __init__(self,data,way,shot,query,episode):
        super().__init__()
        self.data=data
        self.way=way
        self.shot=shot
        self.query=query
        self.episode=episode

        self.class_to_indices=self._build_class_index()
        self.classes=list(self.class_to_indices.keys())
        

    def _build_class_index(self):
        class_index={}

        targets=self.data.dataset.targets

        labels = [self.data.dataset.targets[i] for i in self.data.indices]
        


        for indexofsubset, indexoforiginal in enumerate(self.data.indices):
            label=targets[indexoforiginal]
            if label not in class_index:
                class_index[label]=[]
            class_index[label].append(indexofsubset)

        return class_index
        
    def __len__(self):
            return self.episode
        
    def __getitem__(self, idx):
        #print('hello')

        #print(f"Total available classes: {len(self.classes)}, requested way: {self.way}")


        selected_class=random.sample(self.classes,self.way)

        support_images, support_labels=[],[]
        query_images, query_labels=[],[]


        label_map={class_name: i for i, class_name in enumerate(selected_class)}

        for class_name in selected_class:
            all_indices_for_class=self.class_to_indices[class_name]

            selected_index=random.sample(all_indices_for_class,self.shot+self.query)

            support_index=selected_index[:self.shot]
            query_index=selected_index[self.shot:]

            for i in support_index:
                image,_=self.data[i]
                support_images.append(image)
                support_labels.append(torch.tensor(label_map[class_name]))
                
            for i in query_index:
                image,_=self.data[i]
                query_images.append(image)
                query_labels.append(torch.tensor(label_map[class_name]))
            
        return(
            torch.stack(support_images),
            torch.stack(support_labels),
            torch.stack(query_images),
            torch.stack(query_labels)
        )

In [20]:
def compute_prototypes(support_embeddings,support_labels,way):
    embedding_dimensions=support_embeddings.size(-1)
    prototypes=torch.zeros(way,embedding_dimensions).to(support_embeddings.device)

    for c in range(way):
        class_mask=(support_labels==c)
        class_embeddings=support_embeddings[class_mask]
        prototypes[c]=class_embeddings.mean(dim=0)
    return prototypes

def classify_queries(prototypes,query_embeddings):
    n_query=query_embeddings.size(0)
    way=prototypes.size(0)

    query_exp=query_embeddings.unsqueeze(1).expand(n_query,way,-1)
    prototypes_exp=prototypes.unsqueeze(0).expand(n_query,way,-1)

    distances=torch.sum((query_exp-prototypes_exp)**2,dim=2)

    logits=-distances
    return logits


In [21]:
import torch.optim as optim

few_dataset=create_dataset(
    data=train_data,
    way=ways,
    shot=shots,
    query=queries,
    episode=200
)

In [22]:
few_dataloader=DataLoader(
    few_dataset,
    #batch_size=1,
    shuffle=True
)

In [23]:
import torchvision.models as models
vgg=models.vgg16(pretrained=True)

In [24]:
class VGGEmbedding(nn.Module):
    def __init__(self):
        super().__init__()


        # ##Code from the paper
        # features_list = list(vgg.features.children())
        # # Insert DropBlock after MaxPool at index 16
        # features_list.insert(17, DropBlock2D(block_size=block_size, drop_prob=drop_prob))
        # # Insert DropBlock after MaxPool at index 23 (now 24 due to previous insertion)
        # features_list.insert(24, DropBlock2D(block_size=block_size, drop_prob=drop_prob))
        # ##END


        self.features=vgg.features
        self.avgpool=vgg.avgpool

        self.classifier=nn.Sequential(*list(vgg.classifier.children())[:-1])

    def forward(self,x):
        x=self.features(x)
        x=self.avgpool(x)
        x=torch.flatten(x,1)
        x=self.classifier(x)
        return x

In [25]:
model=VGGEmbedding()

for param in model.features.parameters():
    param.requires_grad=False

model.classifier[3]=nn.Linear(model.classifier[3].in_features,256)

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=model.to(device)


optimizer=optim.Adam(model.parameters(),lr=1e-4)
loss_fn=nn.CrossEntropyLoss()

epochs=20


In [26]:
print("training on class :",train_list)

training on class : [6, 2, 1, 3, 8, 9, 5, 7]


In [27]:

for epoch in range(epochs):
    model.train()
    total_loss, total_correct, total_queries= 0,0,0

    for episode in few_dataloader:
        support_images, support_labels, query_images, query_labels=episode
        support_images=(support_images.squeeze(0)).to(device)
        query_images=(query_images.squeeze(0)).to(device)
        support_labels=(support_labels.view(-1)).to(device)
        query_labels=(query_labels.view(-1)).to(device)

        optimizer.zero_grad()
        support_embeddings=model(support_images)
        query_embeddings=model(query_images)

        n_way=torch.unique(support_labels).size(0)
        prototypes=compute_prototypes(support_embeddings,support_labels,n_way)
        logits=classify_queries(prototypes,query_embeddings)
        loss=loss_fn(logits,query_labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
        preds=torch.argmax(logits,dim=1)
        total_correct+=(preds==query_labels).sum().item()
        total_queries+=query_labels.size(0)
    
    avg_loss=total_loss/len(few_dataloader)
    accuracy=(total_correct/total_queries)*100
    print("Epoch:",epoch+1,"-------------","Loss=",avg_loss,"Acccuracy=",accuracy)

Epoch: 1 ------------- Loss= 1.730743027627468 Acccuracy= 50.160000000000004
Epoch: 2 ------------- Loss= 1.345016317665577 Acccuracy= 53.2
Epoch: 3 ------------- Loss= 1.2752484428882598 Acccuracy= 54.339999999999996
Epoch: 4 ------------- Loss= 1.060612289607525 Acccuracy= 63.339999999999996
Epoch: 5 ------------- Loss= 0.9476573210209608 Acccuracy= 68.04
Epoch: 6 ------------- Loss= 0.8292621003091335 Acccuracy= 73.4
Epoch: 7 ------------- Loss= 0.7213423486426472 Acccuracy= 76.4
Epoch: 8 ------------- Loss= 0.661563436165452 Acccuracy= 78.86
Epoch: 9 ------------- Loss= 0.5826441954821349 Acccuracy= 81.32000000000001
Epoch: 10 ------------- Loss= 0.5152879178337753 Acccuracy= 83.62
Epoch: 11 ------------- Loss= 0.5318558346340433 Acccuracy= 82.82000000000001
Epoch: 12 ------------- Loss= 0.5072007673978806 Acccuracy= 84.26
Epoch: 13 ------------- Loss= 0.5137435742653906 Acccuracy= 84.08
Epoch: 14 ------------- Loss= 0.49117093347012997 Acccuracy= 84.94
Epoch: 15 ------------- Loss

In [28]:
test_dataset=create_dataset(
    data=test_data,
    way=ways,
    shot=shots,
    query=queries,
    episode=200
)

In [29]:
test_dataloader=DataLoader(
    test_dataset,
    shuffle=True
)

In [30]:
print("testing on class :",test_list)

testing on class : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [31]:
model.eval()
total_correct, total_queries= 0,0
with torch.no_grad():
    for episode in test_dataloader:
        support_images, support_labels, query_images, query_labels=episode
        support_images=(support_images.squeeze(0)).to(device)
        query_images=(query_images.squeeze(0)).to(device)
        support_labels=(support_labels.view(-1)).to(device)
        query_labels=(query_labels.view(-1)).to(device)

        support_embeddings=model(support_images)
        query_embeddings=model(query_images)

        n_way=torch.unique(support_labels).size(0)
        prototypes=compute_prototypes(support_embeddings,support_labels,n_way)
        logits=classify_queries(prototypes,query_embeddings)
        
        preds=torch.argmax(logits,dim=1)
        total_correct+=(preds==query_labels).sum().item()
        total_queries+=query_labels.size(0)
    
    #avg_loss=total_loss/len(few_dataloader)
    accuracy=(total_correct/total_queries)*100
    print("Loss=",avg_loss,"Acccuracy=",accuracy)

Loss= 0.39484588041668756 Acccuracy= 89.48


In [38]:
strict_test_dataset=create_dataset(
    data=strict_test_data,
    way=strict_ways,
    shot=shots,
    query=queries,
    episode=200
)

In [39]:
strict_test_dataloader=DataLoader(
    strict_test_dataset,
    #batch_size=1,
    shuffle=True
)

In [40]:
print("testing on class :",strict_test_list)

testing on class : [0, 4]


In [41]:
model.eval()
total_correct, total_queries= 0,0
with torch.no_grad():
    for episode in strict_test_dataloader:
        support_images, support_labels, query_images, query_labels=episode
        support_images=(support_images.squeeze(0)).to(device)
        query_images=(query_images.squeeze(0)).to(device)
        support_labels=(support_labels.view(-1)).to(device)
        query_labels=(query_labels.view(-1)).to(device)

        support_embeddings=model(support_images)
        query_embeddings=model(query_images)

        n_way=torch.unique(support_labels).size(0)
        prototypes=compute_prototypes(support_embeddings,support_labels,n_way)
        logits=classify_queries(prototypes,query_embeddings)
        
        preds=torch.argmax(logits,dim=1)
        total_correct+=(preds==query_labels).sum().item()
        total_queries+=query_labels.size(0)
    
    #avg_loss=total_loss/len(few_dataloader)
    accuracy=(total_correct/total_queries)*100
    print("Loss=",avg_loss,"Acccuracy=",accuracy)

Loss= 0.39484588041668756 Acccuracy= 79.5


**Stable Protypical Network**

In [42]:
import torch
import gc

# Delete all unused objects
gc.collect()

# Empty PyTorch CUDA cache
torch.cuda.empty_cache()

In [43]:
from dropblock import DropBlock2D

In [45]:
import torchvision.models as models
vgg=models.vgg16(pretrained=True)
class VGGEmbedding(nn.Module):
    def __init__(self,drop_prob=0.3, block_size=5):
        super().__init__()


        ##Code from the paper
        features_list = list(vgg.features.children())
        # Insert DropBlock after MaxPool at index 16
        features_list.insert(17, DropBlock2D(block_size=block_size, drop_prob=drop_prob))
        # Insert DropBlock after MaxPool at index 23 (now 24 due to previous insertion)
        features_list.insert(24, DropBlock2D(block_size=block_size, drop_prob=drop_prob))
        ##END


        self.features=vgg.features
        self.avgpool=vgg.avgpool

        self.classifier=nn.Sequential(*list(vgg.classifier.children())[:-1])

    def forward(self,x):
        x=self.features(x)
        x=self.avgpool(x)
        x=torch.flatten(x,1)
        x=self.classifier(x)
        return x

model=VGGEmbedding()

for param in model.features.parameters():
    param.requires_grad=False

model.classifier[3]=nn.Linear(model.classifier[3].in_features,256)

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=model.to(device)


optimizer=optim.Adam(model.parameters(),lr=1e-4)
loss_fn=nn.CrossEntropyLoss()

epochs=20

#From code for SPN
n_times=5
alpha=0.1

for epoch in range(epochs):
    model.train()
    total_loss, total_correct, total_queries= 0,0,0

    for episode in few_dataloader:
        support_images, support_labels, query_images, query_labels=episode
        support_images=(support_images.squeeze(0)).to(device)
        query_images=(query_images.squeeze(0)).to(device)
        support_labels=(support_labels.view(-1)).to(device)
        query_labels=(query_labels.view(-1)).to(device)


        #For montecarlopass
        all_ce_losses = []
        all_query_logits = []

        for _ in range(n_times):

            #optimizer.zero_grad()
            support_embeddings=model(support_images)
            query_embeddings=model(query_images)

            n_way=torch.unique(support_labels).size(0)
            prototypes=compute_prototypes(support_embeddings,support_labels,n_way)
            logits=classify_queries(prototypes,query_embeddings)

            ce_loss=loss_fn(logits,query_labels)
            all_ce_losses.append(ce_loss)
            all_query_logits.append(logits)
        
        total_ce_loss= torch.stack(all_ce_losses).mean()

        stacked_logits=torch.stack(all_query_logits)
        stacked_probs=torch.softmax(stacked_logits,dim=1)

        true_class_probs=stacked_probs[:, torch.arange(len(query_labels)),query_labels]
        variance_loss=torch.std(true_class_probs,dim=0).mean()

        total_combined_loss=total_ce_loss+alpha*variance_loss

        optimizer.zero_grad()
        total_combined_loss.backward()
        optimizer.step()

        mean_logits=stacked_logits.mean(dim=0)
        total_loss+=total_combined_loss.item()
        preds=torch.argmax(logits,dim=1)
        total_correct+=(preds==query_labels).sum().item()
        total_queries+=query_labels.size(0)
    
    avg_loss=total_loss/len(few_dataloader)
    accuracy=(total_correct/total_queries)*100
    print("Epoch:",epoch+1,"-------------","Loss=",avg_loss,"Acccuracy=",accuracy)




# test_dataset=create_dataset(
#     data=test_data,
#     way=ways,
#     shot=shots,
#     query=queries,
#     episode=200
# )
# test_dataloader=DataLoader(
#     test_dataset,
#     shuffle=True
# )

model.train()
total_correct, total_queries= 0,0
with torch.no_grad():
    for episode in test_dataloader:
        support_images, support_labels, query_images, query_labels=episode
        support_images=(support_images.squeeze(0)).to(device)
        query_images=(query_images.squeeze(0)).to(device)
        support_labels=(support_labels.view(-1)).to(device)
        query_labels=(query_labels.view(-1)).to(device)

        stacked_logits=[]

        for _ in range(n_times):

            support_embeddings=model(support_images)
            query_embeddings=model(query_images)

            n_way=torch.unique(support_labels).size(0)
            prototypes=compute_prototypes(support_embeddings,support_labels,n_way)
            logits=classify_queries(prototypes,query_embeddings)

            stacked_logits.append(logits)
        
        mean_logits=torch.stack(stacked_logits).mean(dim=0)
        preds=torch.argmax(mean_logits,dim=1)
        total_correct+=(preds==query_labels).sum().item()
        total_queries+=query_labels.size(0)

        torch.cuda.empty_cache()
    
    #avg_loss=total_loss/len(few_dataloader)
    accuracy=(total_correct/total_queries)*100
    print("Loss=",avg_loss,"Acccuracy=",accuracy)


    entropy = -(torch.softmax(mean_logits, dim=1) * torch.log_softmax(mean_logits, dim=1)).sum(dim=1).mean()
    print("Mean Predictive Entropy =", entropy.item())


Epoch: 1 ------------- Loss= 1.2163646990060806 Acccuracy= 60.540000000000006
Epoch: 2 ------------- Loss= 0.6792525565624237 Acccuracy= 78.58000000000001
Epoch: 3 ------------- Loss= 0.4883936108648777 Acccuracy= 84.36
Epoch: 4 ------------- Loss= 0.4532578492164612 Acccuracy= 87.03999999999999
Epoch: 5 ------------- Loss= 0.36844709813594817 Acccuracy= 88.68
Epoch: 6 ------------- Loss= 0.318657393604517 Acccuracy= 90.38000000000001
Epoch: 7 ------------- Loss= 0.3201307491585612 Acccuracy= 91.3
Epoch: 8 ------------- Loss= 0.2834145724773407 Acccuracy= 92.12
Epoch: 9 ------------- Loss= 0.2597010859288275 Acccuracy= 92.84
Epoch: 10 ------------- Loss= 0.2431825573183596 Acccuracy= 93.16
Epoch: 11 ------------- Loss= 0.22223338982090354 Acccuracy= 94.0
Epoch: 12 ------------- Loss= 0.22846161099150777 Acccuracy= 93.89999999999999
Epoch: 13 ------------- Loss= 0.18372867686674 Acccuracy= 94.74000000000001
Epoch: 14 ------------- Loss= 0.17523603992536663 Acccuracy= 95.16
Epoch: 15 ---

In [ ]:
model.train()
total_correct, total_queries= 0,0
with torch.no_grad():
    for episode in strict_test_dataloader:
        support_images, support_labels, query_images, query_labels=episode
        support_images=(support_images.squeeze(0)).to(device)
        query_images=(query_images.squeeze(0)).to(device)
        support_labels=(support_labels.view(-1)).to(device)
        query_labels=(query_labels.view(-1)).to(device)

        stacked_logits=[]

        for _ in range(n_times):

            support_embeddings=model(support_images)
            query_embeddings=model(query_images)

            n_way=torch.unique(support_labels).size(0)
            prototypes=compute_prototypes(support_embeddings,support_labels,n_way)
            logits=classify_queries(prototypes,query_embeddings)

            stacked_logits.append(logits)
        
        mean_logits=torch.stack(stacked_logits).mean(dim=0)
        preds=torch.argmax(mean_logits,dim=1)
        total_correct+=(preds==query_labels).sum().item()
        total_queries+=query_labels.size(0)

        torch.cuda.empty_cache()
    
    #avg_loss=total_loss/len(few_dataloader)
    accuracy=(total_correct/total_queries)*100
    print("Loss=",avg_loss,"Acccuracy on", len(strict_test_list),"Class =",accuracy)


    entropy = -(torch.softmax(mean_logits, dim=1) * torch.log_softmax(mean_logits, dim=1)).sum(dim=1).mean()

Loss= 0.1253836838528514 Acccuracy on 2  Class = 78.14999999999999
